# TaskManager

The `TaskManager` is responsible for the submission of the tasks (creation of the submission script, initialization of the shell environment) as well as for the optimization of the parameters used for the parallel runs (number of MPI processes, number of OpenMP threads, automatic parallelization with ABINIT `autoparal` feature). 

The configuration file for the `TaskManager` is written in YAML (a good introduction to the YAML syntax can be found at http://yaml.org/spec/1.1/#id857168).
See also http://www.yaml.org/refcard.html for the YAML reference card.
A typical example is reported below:

`qtype` specifies the queue resource manager (Slurm in this example). `qparams` is a dictionary with the parameters 
passed to the resource manager. 
We use the *normalized* version of the options i.e dashes in the official name of the parameter are replaced by  underscores  (for the list of supported options see ...)
`modules` is the list of modules to load, while `shell_env` allows the user to specify or to modfiy the values of the environment variables.
The `policy` section governs the automatic parallelization of the run: in this case abipy will use the `autoparal` features of abinit to determine an optimal configuration with **maximum** `max_ncpus` MPI nodes. Setting autoparal to 0 disables the automatic parallelization. **Other values of autoparal are not supported**.
One can put this configuration file either in the configuration directory `$HOME/.abinit/abipy` or in the current working directory (the latter has precedence over the global configuration file located in `$HOME/.abinit/abipy`).
The `TaskManager` can then be easily initialized by calling the class method `from_user_config`  

In [1]:
from abipy import abilab 
manager = abilab.TaskManager.from_user_config()
print(manager)

[Qadapter 0]
ShellAdapter:localhost
Hardware:
   num_nodes: 1, sockets_per_node: 1, cores_per_socket: 2, mem_per_node 4096,
Qadapter selected: 0


In some cases, you may want to enforce some constraint on the "optimal" configuration. For example, you may want to select only those configurations whose parallel efficiency is greater than 0.7 and whose number of MPI nodes is divisible
by 4. One can easily enforce this constraint via the `condition` dictionary whose syntax is similar to the one used in `mongodb`

```json
policy:
    autoparal: 1
    max_ncpus: 10
    condition: {$and: [ {"efficiency": {$gt: 0.7}}, {"tot_ncpus": {$divisible: 4}} ]}
```

The parallel efficiency is defined as $\epsilon = \dfrac{T_1}{T_N * N}$ where $N$ is the number of MPI processes and $T_j$ is the wall time 
needed to complete the calculation with $j$ MPI processes. For a perfect scaling implementation $\epsilon$ is equal to one.
The parallel speedup with N processors is given by $S = T_N / T_1$.
Note that `autoparal = 1` will automatically change your `job.sh` script as well as the input file so that we can run the job in parallel with the optimal configuration required by the user. For example, you can use `paral_kgb` in GS calculations and `abipy` will automatically set the values of `npband`, `npfft`, `npkpt` ... for you! 
Note that if no configuration fulfills the given condition, abipy will use the optimal configuration that leads to the highest parallel speedup (not necessarily the most efficient one).

The complete list of options supported by the `TaskManager` with slurm can be 
retrieved with the command:

In [6]:
!abirun.py . doc_manager slurm


# TaskManager configuration file (YAML Format)

policy:
    # Dictionary with options used to control the execution of the tasks.

qadapters:
    # List of qadapters objects (mandatory)
    -  # qadapter_1
    -  # qadapter_2

db_connector:
    # Connection to MongoDB database (optional)

batch_adapter:
    # Adapter used to submit flows with batch script. (optional)

##########################################
# Individual entries are documented below:
##########################################

policy: 
    autoparal:                # (integer). 0 to disable the autoparal feature (DEFAULT: 1 i.e. autoparal is on)
    condition:                # condition used to filter the autoparal configurations (Mongodb-like syntax).
                              # DEFAULT: empty i.e. ignored.
    vars_condition:           # Condition used to filter the list of ABINIT variables reported by autoparal
                              # (Mongodb-like syntax). DEFAULT: empty i.e. ignored.
    frozen_time